1. Choose runtime type as T4 GPU

In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


CUDA is available.
CUDA version: 12.1
GPU: Tesla T4


2. Upload the model

In [ ]:
%cd /content
!git clone --recursive https://github.com/vaishvijs13/se101-model
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

3. Add folder of images from Drive (change folder name to 'input' and move this subfolder to a larger folder, ie. 'colmap')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

4. COLMAP dependencies:

In [4]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-regex-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    libsuitesparse-dev \
    libgflags-dev \
    libpng-dev \
    libjpeg-dev \
    libtiff-dev \
    libxxf86vm1 \
    libxxf86vm-dev \
    libxi-dev \
    libxrandr-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-regex-dev is already the newest version (1.74.0.3ubuntu7).
libboost-regex-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libjpeg-dev is already the newest version (8c-2ubuntu10).
libjpeg-dev set to manually installed.
libpng-dev is already the newest version (1.6.37-3build5).
libxxf86vm1 is already the newest version (1:1.1.4-1build3).
libxxf86vm1 set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manu

5. Build COLMAP


In [ ]:
# /content for colab
%cd /content

# clone if does not already exist
!if [ ! -d "colmap" ]; then git clone https://github.com/colmap/colmap.git; fi
%cd colmap
!mkdir -p build
%cd build
!cmake ..
!make -j$(nproc)
!sudo make install

# confirm installed/in path
!colmap --help || echo "colmap not in path or installation failed"


/content
Cloning into 'colmap'...
remote: Enumerating objects: 23160, done.
remote: Counting objects: 100% (2085/2085), done.
remote: Compressing objects: 100% (1301/1301), done.
remote: Total 23160 (delta 1245), reused 1431 (delta 775), pack-reused 21075 (from 1)
Receiving objects: 100% (23160/23160), 18.87 MiB | 23.42 MiB/s, done.
Resolving deltas: 100% (17732/17732), done.
/content/colmap
/content/colmap/build
-- Enabling LSD support
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found FreeImage
--   Includes : /usr/include
--   L

6. Convert the dataset using COLMAP <br />

In [ ]:
%cd /content/gaussian-splatting
!python convert.py -s /content/gaussian-splatting/colmap

Streaming output truncated to the last 5000 lines.
I1129 20:56:20.904907 20206 incremental_pipeline.cc:390] Registering image #136 (25)
I1129 20:56:20.904942 20206 incremental_pipeline.cc:393] => Image sees 313 / 1892 points
I1129 20:56:21.789183 20206 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I1129 20:56:30.514626 20206 incremental_pipeline.cc:390] Registering image #650 (26)
I1129 20:56:30.514660 20206 incremental_pipeline.cc:393] => Image sees 313 / 2179 points
I1129 20:56:31.470707 20206 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
W1129 20:56:32.326922 20206 levenberg_marquardt_strategy.cc:115] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W1129 20:56:32.427307 20206 levenberg_marquardt_strategy.cc:115] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W1129 20:56:32.453503 20206 levenberg_marquar

7. Train the converted dataset

In [ ]:
!python train.py -s /content/gaussian-splatting/colmap --iterations 30000 --save_iterations 10000 20000 30000

In [ ]:
file_path = '/content/gaussian-splatting/output/7d570830-6/point_cloud/iteration_30000/point_cloud.ply'  # replace with the actual path
drive_path = '/content/drive/My Drive/stc_model.ply'
!cp "{file_path}" "{drive_path}"
print(f"File saved to Google Drive: {drive_path}")